
# ChatGPT Prompt Engineering For Developers

## 1 | Introduction

For developers, the power of LLM (Large Language Models) comes in the ability to deliver rich experiences by making API calls to an LLM to achieve tasks like summarization, inference, transformation, and expansion - of text inputs. In this course, we learn to build a chat bot using API calls to LLM.

There are 2 types of LLM: _Base LLM_ trained to predict the next word, and _Instruction-Tuned LLM_ trained to follow instructions. The latter uses Reinforcement Learning with Human Feedback (RLHF) and is trained to be _Helpful, Honest, Harmless_. This course recommends we prioritize focus on Instruction-Tuned LLM (IT-LLM) and teaches best practices for doing so.

Think of an IT-LLM as a smart-friend but not a mind reader. In other words, when an IT-LLM fails to deliver an expected result, it might be because the _instructions it was given_ were not representative. Some guidance:
 - Be Specific: Was I clear in what I wanted the focus of the result to be?
 - Set Tone: Was I clear about the tone (conversational, formal, social etc.) to use in writing?
 - Give Context: Can I provide explicit snippets of text or references that can help specialize this further?

References:
1. [OpenAI Cookbook: Recipes](https://github.com/openai/openai-cookbook)
2. [OpenAI Docs: Tutorials](https://platform.openai.com/docs/tutorials)
3. [OpenAI Gallery: Examples](https://platform.openai.com/examples)
4. [GPT Best Practices: 6 Strategies](https://platform.openai.com/docs/guides/gpt-best-practices/six-strategies-for-getting-better-results)

✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨

## 2 | Guidelines for Prompting

2 Principles for Prompt Writing:
 1. Write clear and specific prompts.
 2. Give the model time to think.

👉🏽👉🏽👉🏽 Note: Since this course was released, there has been a lot more progress in GPT models.
 1. Official GPT Best Practices Page: https://platform.openai.com/docs/guides/gpt-best-practices
 2. Now lists 6 strategies for improving results, including:
    1. Write Clear Instructions
    2. Provide Reference Text
    3. Split Complex Tasks Into Simpler Subtasks
    4. Give GPTs Time To Think
    5. Use External Tools
    6. Test Changes Systematically
 3. Each strategy comes with its own list of tactics.
👉🏽👉🏽👉🏽 For this Notebook we follow only the DeepLearning.AI exercises (2 strategies)


### 2.1 | Setup for OpenAI Exercises

Adding this section to document how we configure our Codespaces to support this exercises:
1. Install `openai` Python library. 
    - Use `pip install openai` to do this temporarily, in current container
    - Update _requirements.txt_ to add `openai` to do this permanently (and rebuild current container).
2. Set the OPENAI_API_KEY environment variable to your OpenAI key.
    - First, get an OpenAI account. Create new secret key [under API keys](https://platform.openai.com/account/api-keys) - and copy it.
3. Use the OpenAI key in GitHub Codespaces 👉🏽 by creating a Codespaces Secret
    - Now [create a Codespaces secret](https://github.com/settings/codespaces) with name OPENAI_API_KEY
    - Paste the OpenAI secret key from step 2 into the _Value_ field.
    - Select repositories that can access to this key (e.g., this one) and "Add secret"
    - The secret is now _automatically exported_ [as an env var in user's terminal](https://docs.github.com/en/codespaces/managing-your-codespaces/managing-encrypted-secrets-for-your-codespaces#using-secrets)
    - Rebuild container to enforce new secrets - test in terminal with `echo $OPENAI_API_KEY`
    - Note: this works in GitHub Codespaces (cloud env) but not in Docker Desktop (local env)
4. Use the OpenAI key in Docker Desktop 👉🏽 
    - Check out [discussion on Docker secrets](https://github.com/microsoft/vscode-remote-release/issues/4841) for updates
    - For now, I did the following as a quick workaround:
        - Create or open `.env` file in root folder. Add line with  `OPENAI_API_KEY=sk-xxxxxxxx` and save it.
        - Restart your kernel. The code below should work as is (load_dotenv reads .env file for env var by default)
        - ‼️ Important: Don't forget to add ".env" to your .gitignore so it does not check in your key/secrets

✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨

### 2.2 | Use OpenAI, Create Helper Function

In [ ]:
import openai
import os

# Expects OPENAI_API_KEY in env variables 
# For GitHub Codespaces: set this as Codespaces secret => shows up as env var in OS
# For Docker Desktop: create a .env file (and .gitignore it explicitly to be safe) => shows up as env var from load_dotenv
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# Note that we can set different env variables to different OPENAI keys and just map the right one to openai.api_key here
# Example: have both OPENAI_API_KEY (for OpenAI) and AOAI_API_KEY (for Azure OpenAI) as options 
openai.api_key  = os.getenv('OPENAI_API_KEY')

# Helper function simplifies your call to the Chat completions API at:
# https://platform.openai.com/docs/guides/gpt/chat-completions-api
# The API takes {model, messages[ ]}  => returns model-generated message as output
# Messages are an array of message objects => each message has role (system, user or assistant) + content
#   System message - set assistant behavior
#   User messages - are the prompts (requests) from user
#   Assistant messages - can be written by you (e.g., examples of behavior) or store prior assistant responses
# Conversation can be as short as 1 message => or many back-and-forth turns
#      Conversation formatted with a system message first
#      Conversation then continues with alternating user (prompt) and assistant (respond) messages 
# The LLM is stateless (has no memory of the past conversation) 
#    so every request must current relevant history as messages
#    since there is a token limit for models, you need tactics to shorten history if it is too long
#    See: https://platform.openai.com/docs/guides/gpt-best-practices/tactic-for-dialogue-applications-that-require-very-long-conversations-summarize-or-filter-previous-dialogue
#
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

### 2.3 | Apply Prompting Principles 
1. Write clear and specific instructions. 
2. Give model time to think

In [ ]:
# Write Clear and Specific Instructions

# Tactic 1: 
#   Use delimiters
# Examples: 
#   " ", ---, ` `, < > , <tag></tag>
# How this helps:
#   Provides clear context for the focus of the prompt
#   Protects against prompt injection where client adds _instructions_ into the text (maliciously) and the since LLM is instruction-tuned, it interprets those as actions to take (rather than as inputs to original instruction)
#   Using delimiters explicitly reinforces to LLM that all text provided by client should be treated as input to the current instruction (and not be interpreted)
#   

## Example text
text = f"""
You should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
This will guide the model towards the desired output, \ 
and reduce the chances of receiving irrelevant \ 
or incorrect responses. Don't confuse writing a \ 
clear prompt with writing a short prompt. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
"""

## Set the prompt
prompt = f"""
Summarize the text delimited by triple backticks \ 
into a single sentence.
```{text}```
"""

## Run the prompt
response = get_completion(prompt)
print(response)

In [ ]:
# Write Clear and Specific Instructions

# Tactic 2: 
#   Ask for structured output like HTML or JSON

## Example:
prompt = f"""
Generate a list of three made-up book titles along \ 
with their authors and genres. 
Provide them in JSON format with the following keys: 
book_id, title, author, genre.
"""
response = get_completion(prompt)
print(response)

## Benefit:
## Output is printed in JSON format which can be read directly into a Python dictionary for other analysis